In [1]:
import numpy as np
from ipywidgets import interact, HTML
import random
import pdir


from sidepanel import SidePanel

import ipywidgets as widgets

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters.filters import * 
from ipyregulus.filters.monitor import Monitor
from ipyregulus.tree.has_tree import *
from ipyregulus.alg.view import *


from regulus.topo import morse_smale, Regulus
from regulus.data.data import Data
from regulus.measures import *
from regulus.models import *
import pandas as pd

In [2]:
samples = 200

# from test_functions import *
# test_function = gerber_different_heights
# X = np.random.rand(samples,2)
# Y = test_function(X)
# all_pts = np.hstack((X,np.expand_dims(Y, axis=1)))

# myfun = test_function
from Hartmann import create, calc_Hartmanny

all_pts = create(samples)
myfun = calc_Hartmanny



In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics.pairwise import cosine_similarity
from refine_topo import Refine
from resample import Resample


def create_reg(all_pts, k=8):
    cols = ['x'+str(i) for i in range(all_pts.shape[1])]

    cols[-1] = 'y'

    df = pd.DataFrame(all_pts, columns=cols)

    pts = Data.from_pts(df)

    # Normalize here  
    
    #pts.normalize()

    print(pts)
    regulus = morse_smale(pts, knn=k)#**kwargs)

    regulus.add_attr('linear', linear_model)
    regulus.add_attr('fitness', fitness)
    regulus.add_attr('relative_fitness', relative_fitness)

    regulus.tree.add_attr('parent_fitness', parent_fitness)
    regulus.tree.add_attr('child_fitness', child_fitness)
    regulus.tree.add_attr('size', node_size)
    regulus.tree.add_attr('rel_size', node_relative_size)
    regulus.tree.add_attr('span', node_span)
    return regulus



def mymeasure(node):
    attr="id"
    if hasattr(node, attr) and hasattr(node,'parent') and hasattr(node.parent,attr):
        cid, pid = getattr(node, attr), getattr(node.parent, attr)
        if cid >=0 and pid>=0: #check both are valid partitions

            cx, cy = node.data.x.values, node.data.y.values
            px, py = node.parent.data.x.values, node.parent.data.y.values
            
            # Linear coeff for both and similarity 
            creg, preg = LinearRegression().fit(cx, cy), \
                        LinearRegression().fit(px, py)

            # fitness
            cscore, pscore = creg.score(cx, cy), preg.score(px, py)
            # sensitivity
            ccoef, pcoef = creg.coef_, preg.coef_
            # intercept
            cint, pint = creg.intercept_, preg.intercept_

            return cosine_similarity(ccoef.reshape(1, -1), pcoef.reshape(1, -1))[0][0]
    return -1  


def regenerate(all_pts, partitions, test_function, pids, step=5):
    dims = all_pts.shape[1]-1
    new_x = np.array([], dtype=np.float64).reshape(0,dims)
    r = Resample()
    
    for ind in pids:
        cur_x = partitions[ind].x.values
        # print("CURX = ", cur_x)
        cur_y = partitions[ind].y.values.reshape(-1,1)
        # print("CURY = ", cur_y)

        cur_p = np.hstack((cur_x, cur_y))
        
        r.resample_data(cur_p, step)
        out = r.newsamples
        new_x = np.vstack((new_x, out))
    new_y = test_function(new_x).reshape(-1,1)
    new_pts = np.hstack((new_x,new_y))
    all_pts2 = np.vstack((all_pts, new_pts))
    return all_pts2


In [6]:
regulus = create_reg(all_pts, k = 10)

partitions = list(regulus.partitions())

rt = Refine(0.5, tree=regulus.tree, traverse="BFS", measure=mymeasure, compare="less", return_attr=lambda x:x.id)

pids = rt.search()

print("Partitions to resample", pids)

all_pts = regenerate(all_pts, partitions, myfun, pids, step=5)

E:\Anaconda\envs\CyPlot\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
E:\Anaconda\envs\CyPlot\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


found 9 singles
len(idx)= 278
Partitions to resample [88, 58, 87, 57, 71, 86, 54, 48, 41, 47, 38, 35, 32, 30, 29, 28]


In [31]:
rooooooot = partitions[0]
loc = rooooooot.regulus.pts_loc
idx = [loc[i] for i in range(rooooooot.pts_span[0], rooooooot.pts_span[1]-1)]
idx.extend(rooooooot.minmax_idx)
rooooooot.regulus.pts.x.loc[idx]

,x0,x1
173,0.974846,0.557201
23,0.016288,0.062860
131,0.057463,0.933980
179,0.328222,0.936759
53,0.352961,0.880588
65,0.306847,0.818796
81,0.314088,0.778109
99,0.439329,0.858307
108,0.509882,0.867119
125,0.419620,0.806780


In [32]:
#all_pts2 = regenerate(all_pts, partitions, myfun, pids, step=5)
regulus2 = create_reg(all_pts2, k = 10)

partitions2 = list(regulus2.partitions())

rt2 = Refine(0.4, tree=regulus2.tree, traverse="BFS", measure=mymeasure, compare="less", return_attr=lambda x:x.id)

#pids2 = rt2.search()

#pids.append(24)
#print("Partitions to resample", pids)



E:\Anaconda\envs\CyPlot\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
E:\Anaconda\envs\CyPlot\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


found 7 singles
len(idx)= 223


AttributeError: module 'old_code.refine_topo_' has no attribute 'search'

In [33]:
tree2, data2 = process(regulus2)
treev2 = draw_tree(tree2)
details2 = draw_details(data2)
link_views(treev2, details2)


TreeView(field='span', tree_model=TreeWidget(attrs={'span': {0: 0.6423518529689731, 1: 0.09353991138994455, 2:…

In [33]:
import old_code.refine_topo_ as rt
from old_code.resample_ import get_pts, Resample

dims = all_pts.shape[1]-1
new_x = np.array([], dtype=np.float64).reshape(0,dims)
leafnodes = rt.check_fit(0.4, tree=tree, reg_data = data, raw_data=all_pts)

a = lambda x:x.id
pidds = [a(l) for l in leafnodes]
print(pidds)

cols = ['x'+str(i) for i in range(all_pts.shape[1])]

cols[-1] = 'y'
df = pd.DataFrame(all_pts, columns=cols)

for i in leafnodes:
    ind = i.id
    ind = get_pts(data.partitions[ind],data.pts_loc)
    cur_p = df.loc[ind,:]
    r = Resample(cur_p)
    r.add_samples(5)
    out = r.get_input()
    new_x = np.vstack((new_x, out))
    # May need to rm duplicates 
#print(new_x.shape)    
new_y = test_function(new_x).reshape(-1,1)
#print(new_y.shape)
new_pts = np.hstack((new_x,new_y))

all_pts2 = np.vstack((all_pts, new_pts))



[14, 54, 51, 43]
x =  [[1.62880551e-02 6.28601756e-02]
 [6.48884385e-02 7.77560140e-01]
 [8.18337464e-02 8.84825474e-01]
 [1.72570538e-01 8.41324528e-01]
 [1.77707459e-01 8.32518547e-01]
 [1.13136411e-01 8.61953685e-01]
 [1.91979000e-01 9.80777673e-01]
 [1.89310572e-01 8.83167573e-01]
 [1.74012670e-01 8.42255360e-01]
 [1.10116966e-01 9.78921654e-01]
 [2.70579222e-02 8.05021355e-01]
 [2.79324745e-02 8.27441317e-01]
 [6.54932662e-02 5.90266578e-01]
 [6.43947470e-02 1.07301469e-01]
 [7.58692358e-02 1.33692398e-01]
 [2.30495837e-01 2.06934684e-01]
 [7.14296325e-02 1.64737781e-01]
 [1.84422468e-01 1.56912543e-01]
 [8.77211380e-02 2.62483186e-01]
 [2.29224159e-01 1.11486435e-02]
 [2.13511032e-01 7.14114109e-01]
 [1.25297098e-01 5.93075216e-01]
 [1.85368691e-01 6.35595008e-01]
 [1.17214631e-01 6.61693024e-01]
 [4.42329317e-02 6.93486842e-01]
 [5.98392429e-04 6.76020783e-01]
 [6.96386253e-02 7.02416523e-01]
 [1.60107992e-01 5.24432420e-01]
 [1.54521159e-01 5.44788373e-01]
 [9.81291452e-02 3.91

In [31]:
all_pts2.shape

(225, 3)

In [13]:
np.min(all_pts, axis=0)4

array([-3.83374151e+00, -1.19330488e+00,  4.08570116e-07])

In [10]:
def process(regulus):
    from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget

    tree = TreeWidget(regulus.tree)
    data = DataWidget(data=regulus)
    return tree, data


def draw_tree(tree):
    treev = show_tree(tree)
    w = widgets.Dropdown(
        options=['span', 'fitness', 'parent_fitness', 'child_fitness'],
        value='span',
        description='Attribute:',
        disabled=False,
    )
    panel = SidePanel(title='Fitness')#, side='split-bottom', ref='SidePanel')
    with panel:
        display(w, treev.view, treev.filter)
    return treev


def draw_details(data):
    details = DetailsView(data=data)

    p7 = SidePanel(title='details')
    with p7:
        display(details)
    return details

def link_views(treev, details):
    widgets.link((treev.view, 'details'), (details, 'show'))

In [11]:
tree, data = process(regulus)
treev = draw_tree(tree)
details = draw_details(data)
link_views(treev, details)


TreeView(field='span', tree_model=TreeWidget(attrs={'span': {0: 0.3749545336451027, 1: 0.33219917261226684, 2:…

In [72]:
details.show = list(map(lambda x: x, pids))


# Create resample class, resampling a user specified partition, for User-guided resampling

In [8]:
from resample import get_pts, Resample

# Change the following line to use differennt partition to resample
ind = get_pts(data.partitions[0],data.pts_loc)

selectP = df.loc[ind,:]
r = Resample(selectP,pred_method="bounding_box")

ImportError: cannot import name 'get_pts'

# A line chart for a specified partition, different interactions could be added 

In [29]:
from cyplot import plot
myp = plot(selectP,index = 'y',debug=True, dims=['x0','x1','y'])#ptype='PCA')#, ptype='line'/ PCA. line. dims=[]
myp.enable(['brush_x','brush'])
myp.on('brush_x', r.cb)
myp.on('brush', r.cb)

display(myp)

In [30]:
# from cyplot import plot
myp2 = plot(selectP,index = 'y',debug=True, ptype='PCA')#, ptype='line'/ PCA. line. dims=[]
myp2.enable(['brush_x','brush'])
myp2.on('brush_x', r.cb)
myp2.on('brush', r.cb)
display(myp2)

In [31]:
myp2.enable('x_brush')
display(myp2)

# random generate pts based on user selected range of function value

In [32]:
r.add_samples(5) # user's specified # of samples


# When user adds enough samples, they could use regression to predict the point locations to resample 

In [33]:
# Would scale back if needed 
new_x = r.get_input()
new_y = test_function(new_x).reshape(-1,1)
new_pts = np.hstack((new_x,new_y))



In [34]:
all_pts = np.vstack((all_pts, new_pts))